In [2]:
# read data

from mnist import MNIST
import numpy as np
mndata = MNIST("/Users/jigyayadav/Desktop/Codes/neuralnets253/HW1")
mndata.gz = True
images, labels = mndata.load_training() #Images is a list of 60000 images of 784 dimensions, Labels is a list of 60000 ints
imagesTest, labelsTest = mndata.load_testing()

images = np.array(images)
labels = np.array(labels)
imagesTest = np.array(imagesTest)
labelsTest = np.array(labelsTest)

images_train = images[:200]
images_test = imagesTest[-50:]
labels_train = labels[:200]
labels_test = labelsTest[-50:]
images_train = np.array(images_train)
labels_train = np.array(labels_train)
images_test = np.array(images_test)
labels_test = np.array(labels_test)

# Normalization
images_train = images_train*(1/127.5)
images_test = images_test*(1/127.5)
arrOnes = np.ones_like(images_train)
images_train = images_train-arrOnes
arrOnes = np.ones_like(images_test)
images_test = images_test-arrOnes

# Should this be appended after or before normalization
images_train = np.insert(images_train, 0, 1, axis=1)
images_test = np.insert(images_test, 0, 1, axis=1)

numFeatures = len(images[0, :])

# Divide between validation and training
from sklearn.model_selection import train_test_split
images_train, images_validation, labels_train, labels_validation = train_test_split(images_train, labels_train, test_size=0.10)

In [4]:
labelNums = 10

def labelsToVectForm(labels):
    vectForm = np.zeros((len(labels), labelNums))
    for i in range(len(labels)):
        vectForm[i][labels[i]] = 1
    return vectForm

labels_train = labelsToVectForm(labels_train)
labels_test = labelsToVectForm(labels_test)
labels_validation = labelsToVectForm(labels_validation)

In [17]:
labels_validation.shape

(20, 10)

In [25]:
import math

# Calculate things going forward
from sklearn.metrics import accuracy_score

numInputLayers = 784
numHiddenLayers = 64
numOutputLayers = 10
numExamples = 100

# X is (n*i) and weights is (i*j)
# returns (n*j) vector
def calculateWeightedInput(weights, x):
    return (x.dot(weights))

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def softmaxFunc(prod):
#     for i in range(len(prod)):
#         for j in range(len(prod[i, :])):
#             prod[i][j] = math.exp(prod[i][j])
    prod = np.exp(prod)
    for i in range(len(prod)):
        rowSum = sum(prod[i, :])
        for j in range(len(prod[i, :])):
            prod[i][j] = prod[i][j]/rowSum    
    return prod

# X is (n*i) and weights is (i*j)
# returns (n*j) vector
def sigmoidActivation(weights, x):
    print("In sigmoid activation")
    print(weights.shape)
    print(x.shape)
    prod = calculateWeightedInput(weights, x)
    for i in range(len(prod)):
        for j in range(len(prod[i, :])):
            prod[i][j] = sigmoid(prod[i][j])
    return prod

# X is (n*i) and weights is (i*j)
# returns (n*j) vector
def softmaxActivation(weights, x):
    prod = calculateWeightedInput(weights, x)
    prod = softmaxFunc(prod)
    return prod

# X is (n*i) and weights is (i*j)
# returns (n*j) vector
def YHotEncoding(a):
    b = np.zeros_like(a)
    b[np.arange(len(a)), a.argmax(1)] = 1
    return b

def calculateOutputPerLayer(weights, x):
    outputHiddenLayer = sigmoidActivation(weights[0], x)
    inputSecondLayer = np.insert(outputHiddenLayer, 0, 1, axis=1)
    outputFinalLayer = softmaxActivation(weights[1], inputSecondLayer)
#     outputFinalLayer = findYHotEncoding(outputFinalLayer)
    return [outputHiddenLayer, outputFinalLayer]

# Calculate the derivative of the cost function wrt hidden to output layer weights
def delEWjk(x, y, t, z):
    differencePredictedActual = (y-t)
    # differentiation is j*k array
    differentiation = z.T.dot(differencePredictedActual)
    differentiation = differentiation*(1/len(x))
    return differentiation

# Calculate the derivative of the cost function wrt input to hidden layer weights
def delEWij(x, y, t, g, weights):
    differencePredictedActual = (y-t)
    # summation_k is n*j
    summation_k = (differencePredictedActual.dot(weights.T))
    # calculate g(a_j)*(1-g(a_j))- n*j
    # g_differentiation is (n*j)
    g_differentiation = np.multiply(g, np.ones_like(g)-g)
    prod1 = np.multiply(g_differentiation, summation_k)
    prod2 = x.T.dot(prod1)
    prod2 = prod2*(1/len(x))
    return prod2

# x is n*i vector
# weights is j*k vector
# y and t are (n*k) vector
# z is (n*j) vector
def updateWeightsHiddenToOutput(weights, x, y, t, z, learningRate):
    derivativeE = delEWjk(x, y, t, z)
    updatedWeights = weights - learningRate*derivativeE
    return updatedWeights

# x is (n*i) vector
# y, t is (n*k)
# g is (n*j) vector
# weights is (j*k)
def updateWeightsInputToHidden(weights, x, y, t, g, learningRate):
    derivativeE = delEWij(x, y, t, g, weights)
    updatedWeights = weights - learningRate*derivativeE
    return updatedWeights

def costFunction(x, t, weights):
    sampLen = len(x)
    predProb = calculateOutputPerLayer(weights, x)[1]
#     vectorLabels = labelsToVectForm(t)
    cost = -1*(np.multiply(t, np.log(predProb)))/sampLen
    return [np.sum(cost), cost]

# Calculate derivate of nth example wrt weight_ij
def calculateApproximateDerivative(weights, epsilon, x, y, t, i, j, n, p):
    newWeights = weights
    newWeights[p][i][j] += epsilon
    costFunction1 = costFunction(x, t, newWeights)
    newWeights = weights
    newWeights[p][i][j] -= epsilon
    costFunction2 = costFunction(x, t, newWeights)
    approximateDerivative = (costFunction1[n]-costFunction2[n])/(2*epsilon)
    return approximateDerivative

def checkGradient(weights, epsilon, x, t):
    outputLayers = calculateOutputPerLayer(weights, x)
    y = outputLayers[1]
    
    # Check gradient input to hidden
    for n in range(0, 20000, 2000):
        print("For input to hidden layers")
        for i in range(0, 784, 200):
            for j in range(0, 64, 20):
                summation_k = 0
                for k in range(len(y[0, :])):
                    summation_k += weights[1][j][k]*(y[n][k]-t[n][k])
                actualDerivative = outputLayers[0][n][j]*(1-outputLayers[0][n][j])*x[n][i]*(summation_k)
                approximateDerivative = calculateApproximateDerivative(weights, epsilon, x, y, t, i, j, n, 0)
                diff = abs(actualDerivative-approximateDerivative)
                print("Difference between approximate gradient = ", diff, " ", abs(epsilon*epsilon-diff))
        
        print("For hidden to output layers")
        for j in range(0, 64, 20):
            for k in range(0, 10, 3):
                actualDerivative = outputLayers[0][n][j]*(y[n][k]-t[n][k])
                approximateDerivative = calculateApproximateDerivative(weights, epsilon, x, y, t, i, j, n, 1)
                diff = abs(actualDerivative-approximateDerivative)
                print("Difference between approximate gradient = ", diff, " ", abs(epsilon*epsilon-diff))

def calculateAccuracy(weights, x, t):
    predClass = calculateOutputPerLayer(weights, x)[1]
    predClass = YHotEncoding(predClass)
    return accuracy_score(t, predClass)

def gradientUpdateSingleUpdate(weights, x, t, learningRate):
    outputLayers = calculateOutputPerLayer(weights, x)
    inputSecondLayer = np.insert(outputLayers[0], 0, 1, axis=1)
    updateWtsHiddenToOutput = updateWeightsHiddenToOutput(weights[1], x, outputLayers[1], t, inputSecondLayer, learningRate)
    updateWtsInputToHidden = updateWeightsInputToHidden(weights[0], x, outputLayers[1], t, outputLayers[0], learningRate)
    updatedWeights = [updateWtsInputToHidden, updateWtsHiddenToOutput]
    return updatedWeights

def gradientDescent(numItrs, weights, x, t, learningRate):
    for itr in range(numItrs):
        # Calculate error with the given weights
        currTrainingError = costFunction(x[0], t[0], weights)
        currValidationError = costFunction(x[1], t[1], weights)
        print("Current validation error = ", currValidationError)
        weights = gradientUpdateSingleUpdate(weights, x[0], t[0], learningRate)

checkGradient([weightsIJ, weightsJK], 0.001, images_train, labels_train)
weightsIJ = np.random.rand(785, 64)
weightsJK = np.random.rand(65, 10)
gradientDescent(1000, [weightsIJ, weightsJK], [images_train, images_validation], [labels_train, labels_validation], 0.001)



In sigmoid activation
(785, 64)
(180, 785)
For input to hidden layers
In sigmoid activation
(785, 64)
(180, 785)
In sigmoid activation
(785, 64)
(180, 785)


IndexError: invalid index to scalar variable.